# Earth System Grid Federation Data Access

The Earth System Grid Federation (ESGF) has a search API that can be used by clients to query catalog content matching constraints (see [API documentation](https://github.com/ESGF/esgf.github.io/wiki/ESGF_Search_REST_API)). It's possible to send requests directly to the API using a simple function (see [example](https://esgf2.github.io/cmip6-cookbook/notebooks/foundations/esgf-opendap.html)), but here we'll use a python client named `pyesgf` to interact with the search API and get data from the ESGF THREDDS servers. The following shows examples of typical queries for data. 

If a login username and credentials are required, follow these [instructions](https://esgf-pyclient.readthedocs.io/en/latest/notebooks/examples/logon.html).

In [1]:
from pyesgf.search import SearchConnection

# Create a connection for distributed search on ESGF nodes.
conn = SearchConnection("https://esgf.ceda.ac.uk/esg-search", distrib=True)

# Launch a search query.
# Here we're looking for any variable related to humidity within the CMIP6 SSP2-4.5 experiment.
# Results will be stored in a dictionary with keys defined by the `facets` argument.
ctx = conn.new_context(
    project="CMIP6",
    experiment_id="ssp245",
    query="humidity",
    facets="variable_id,source_id",
)

print("Number of results: ", ctx.hit_count)
print("Variables related to humidity: ")
ctx.facet_counts["variable_id"]

ModuleNotFoundError: No module named 'pyesgf'

In [ ]:
# Now let's look for simulations that have the `hurs` variable and pick the first member.
ctx.constrain(variable_id="hurs", ensemble="r1i1p1f1")
ctx.facet_counts["source_id"]

In [ ]:
# We can now search for datasets corresponding within our search context
results = ctx.search()
r = results[0]
r.dataset_id

In [ ]:
# To get file download links, there's an extra step
file_ctx = r.file_context()
file_ctx.facets = "*"
files = file_ctx.search()
[f.download_url for f in files]

In [ ]:
# The same applies to get an OPENDAP link
agg_ctx = r.aggregation_context()
agg_ctx.facets = "*"
agg = agg_ctx.search()[0]
print(agg.opendap_url)

In [ ]:
# Open the opendap link with xarray
import xarray as xr

ds = xr.open_dataset(agg.opendap_url)
ds